<a href="https://colab.research.google.com/github/manmeet3/Deep_Learning2/blob/master/Asg4/meta-learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Meta Learning
Using Omni glot dataset

In [28]:
# ref: https://github.com/shashankhalo7/Omniglot_meta_learning/blob/master/osl.ipynb

In [29]:
!wget https://github.com/manmeet3/Deep_Learning2/raw/master/Asg4/images_background.zip

--2020-10-18 05:03:00--  https://github.com/manmeet3/Deep_Learning2/raw/master/Asg4/images_background.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/manmeet3/Deep_Learning2/master/Asg4/images_background.zip [following]
--2020-10-18 05:03:00--  https://raw.githubusercontent.com/manmeet3/Deep_Learning2/master/Asg4/images_background.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9464212 (9.0M) [application/zip]
Saving to: ‘images_background.zip.1’

images_background.z 100%[===================>]   9.03M  46.4MB/s    in 0.2s    

2020-10-18 05:03:01 (46.4 MB/s) - ‘images_background.zip.1’ sa

In [30]:
!wget https://github.com/manmeet3/Deep_Learning2/raw/master/Asg4/images_evaluation.zip

--2020-10-18 05:03:01--  https://github.com/manmeet3/Deep_Learning2/raw/master/Asg4/images_evaluation.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/manmeet3/Deep_Learning2/master/Asg4/images_evaluation.zip [following]
--2020-10-18 05:03:01--  https://raw.githubusercontent.com/manmeet3/Deep_Learning2/master/Asg4/images_evaluation.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6462886 (6.2M) [application/zip]
Saving to: ‘images_evaluation.zip.1’

images_evaluation.z 100%[===================>]   6.16M  30.8MB/s    in 0.2s    

2020-10-18 05:03:02 (30.8 MB/s) - ‘images_evaluation.zip.1’ sa

In [31]:
!unzip -q images_background.zip
!unzip -q images_evaluation.zip

replace images_background/Alphabet_of_the_Magi/character01/0709_01.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace images_background/Alphabet_of_the_Magi/character01/0709_02.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
replace images_evaluation/Angelic/character01/0965_01.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [32]:
!ls

data		       images_background.zip.1	images_evaluation.zip.1
images_background      images_evaluation	sample_data
images_background.zip  images_evaluation.zip


In [33]:
import sys
import numpy as np
import pandas as pd
from imageio import imread # from scipy.misc import imread
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

In [60]:
import numpy.random as rng

In [34]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [35]:
# Train contains 30 different languages with 20 instances of 
# each character within each language
train_folder = "images_background/"
# Alphabets for 20 languages
val_folder = "images_evaluation/"
save_path = "data/"

In [36]:
def loadimages(path, n=0):
  '''
  Parameters:
  string:path -> Path of train or test directory

  Return Value:
  tuple:(x, y, language_dictionary)
  '''
  X=[]
  y=[]
  cat_dict = {}
  lang_dict = {}
  curr_y = n

  for alphabet in os.listdir(path):
    print("loading alphabet: " + alphabet)
    lang_dict[alphabet] = [curr_y, None]
    alphabet_path = os.path.join(path, alphabet)
    # Set each letter as its own column
    for letter in os.listdir(alphabet_path):
      cat_dict[curr_y] = (alphabet, letter)
      category_images=[]
      letter_path = os.path.join(alphabet_path, letter)
      # read images in current category
      for filename in os.listdir(letter_path):
        image_path = os.path.join(letter_path, filename)
        image = imread(image_path)
        category_images.append(image)
        y.append(curr_y)
      try:
        X.append(np.stack(category_images))
      # edge case - last one
      except ValueError as e:
        print(e)
        print("error - category_images:", category_images)
      curr_y += 1
      lang_dict[alphabet][1] = curr_y - 1
  y = np.vstack(y)
  X = np.stack(X)
  return X,y,lang_dict

In [37]:
X, y, c = loadimages(train_folder)

loading alphabet: Hebrew
loading alphabet: Greek
loading alphabet: Armenian
loading alphabet: Mkhedruli_(Georgian)
loading alphabet: Latin
loading alphabet: Burmese_(Myanmar)
loading alphabet: Tagalog
loading alphabet: Arcadian
loading alphabet: Futurama
loading alphabet: Anglo-Saxon_Futhorc
loading alphabet: Ojibwe_(Canadian_Aboriginal_Syllabics)
loading alphabet: Braille
loading alphabet: Alphabet_of_the_Magi
loading alphabet: Early_Aramaic
loading alphabet: N_Ko
loading alphabet: Blackfoot_(Canadian_Aboriginal_Syllabics)
loading alphabet: Malay_(Jawi_-_Arabic)
loading alphabet: Japanese_(katakana)
loading alphabet: Tifinagh
loading alphabet: Balinese
loading alphabet: Gujarati
loading alphabet: Asomtavruli_(Georgian)
loading alphabet: Bengali
loading alphabet: Inuktitut_(Canadian_Aboriginal_Syllabics)
loading alphabet: Grantha
loading alphabet: Syriac_(Estrangelo)
loading alphabet: Cyrillic
loading alphabet: Korean
loading alphabet: Japanese_(hiragana)
loading alphabet: Sanskrit


In [38]:
with open(os.path.join(save_path, "train.pickle"), "wb") as f:
  pickle.dump((X,c ), f)

In [39]:
Xval, yval, cval = loadimages(val_folder)

loading alphabet: Malayalam
loading alphabet: Gurmukhi
loading alphabet: Old_Church_Slavonic_(Cyrillic)
loading alphabet: Syriac_(Serto)
loading alphabet: Kannada
loading alphabet: ULOG
loading alphabet: Avesta
loading alphabet: Mongolian
loading alphabet: Glagolitic
loading alphabet: Manipuri
loading alphabet: Ge_ez
loading alphabet: Sylheti
loading alphabet: Atlantean
loading alphabet: Keble
loading alphabet: Angelic
loading alphabet: Tengwar
loading alphabet: Tibetan
loading alphabet: Aurek-Besh
loading alphabet: Oriya
loading alphabet: Atemayar_Qelisayer


In [40]:
with open(os.path.join(save_path, "validate.pickle"), "wb") as f:
  pickle.dump((Xval, cval), f)

In [41]:
def initialize_weights(shape, dtype=None):
  '''
     Initialize CNN layer weights with mean as 0.0 and standard deviation 0f 0.01
  '''
  return np.random.normal(loc = 0.0, scale = 1e-2, size=shape)

In [42]:
def initialize_bias(shape, dtype=None):
  '''
  Initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
  '''
  return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [43]:
def get_siamese_model(input_shape):
  # Define the tensor for the two input images
  left_input = Input(input_shape)
  right_input = Input(input_shape)

  #Convolutional NN
  model = Sequential()
  model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
  model.add(MaxPooling2D())
  model.add(Conv2D(128, (7,7), activation='relu',
                    kernel_initializer=initialize_weights,
                    bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
  model.add(MaxPooling2D())
  model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                    bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
  model.add(MaxPooling2D())
  model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                    bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
  model.add(Flatten())
  model.add(Dense(4096, activation='sigmoid',
                  kernel_regularizer=l2(1e-3),
                  kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
  
  # Generate the encodings for the two images
  encoded_l = model(left_input)
  encoded_r = model(right_input)

  # Add a customized layer to compute the absolute difference between encodings
  l1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
  l1_distance = l1_layer([encoded_l, encoded_r])

  # Add a dense layer with a sigmoid unit to generate similarity score
  prediction = Dense(1, activation='sigmoid', bias_initializer=initialize_bias)(l1_distance)
  
  # Connect the inputs with outputs
  siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)

  return siamese_net

In [44]:
model = get_siamese_model((105, 105, 1)) #105x105 is the dataset image size
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 105, 105, 1) 0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 105, 105, 1) 0                                            
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 4096)         38947648    input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 4096)         0           sequential_3[0][0]    

In [45]:
optimizer = Adam(lr=0.00006)
model.compile(loss="binary_crossentropy", optimizer=optimizer)

In [46]:
with open(os.path.join(save_path, "train.pickle"), "rb") as f:
  (Xtrain, train_classes) = pickle.load(f)

print("Training alphabets: \n")
print(list(train_classes.keys()))

Training alphabets: 

['Hebrew', 'Greek', 'Armenian', 'Mkhedruli_(Georgian)', 'Latin', 'Burmese_(Myanmar)', 'Tagalog', 'Arcadian', 'Futurama', 'Anglo-Saxon_Futhorc', 'Ojibwe_(Canadian_Aboriginal_Syllabics)', 'Braille', 'Alphabet_of_the_Magi', 'Early_Aramaic', 'N_Ko', 'Blackfoot_(Canadian_Aboriginal_Syllabics)', 'Malay_(Jawi_-_Arabic)', 'Japanese_(katakana)', 'Tifinagh', 'Balinese', 'Gujarati', 'Asomtavruli_(Georgian)', 'Bengali', 'Inuktitut_(Canadian_Aboriginal_Syllabics)', 'Grantha', 'Syriac_(Estrangelo)', 'Cyrillic', 'Korean', 'Japanese_(hiragana)', 'Sanskrit']


In [48]:
with open(os.path.join(save_path, "validate.pickle"), "rb") as f:
  (Xval, val_classes) = pickle.load(f)

print("Validation alphabets: \n")
print(list(val_classes.keys()))

Validation alphabets: 

['Malayalam', 'Gurmukhi', 'Old_Church_Slavonic_(Cyrillic)', 'Syriac_(Serto)', 'Kannada', 'ULOG', 'Avesta', 'Mongolian', 'Glagolitic', 'Manipuri', 'Ge_ez', 'Sylheti', 'Atlantean', 'Keble', 'Angelic', 'Tengwar', 'Tibetan', 'Aurek-Besh', 'Oriya', 'Atemayar_Qelisayer']


In [62]:
def get_batch(bach_size,s="train"):
  ''' Create batch of npairs, half same class, half different class'''
  if s == 'train':
    X = Xtrain
    categories = train_classes
  else:
    X = Xval
    categories = val_classes
  n_classes, n_examples, w, h = X.shape

  # randomly sample several classes to use in the batch
  categories = rng.choice(n_classes, size=(batch_size,), replace=False)

  # initialize 2 empty arrays for the input image batch
  pairs = [np.zeros((batch_size, h, w, 1)) for i in range(2)]

  # initialize vector for the targets
  targets=np.zeros((batch_size,))

  # Set 2nd half of batch targets to 1
  targets[batch_size//2:] = 1
  for i in range(batch_size):
    category = categories[i]
    idx_1 = rng.randint(0, n_examples)
    pairs[0][i,:,:,:] = X[category, idx_1].reshape(w, h, 1)
    idx_2 = rng.randint(0, n_examples)

    # pick images of same class for 1st half, different for 2nd half
    if i >= batch_size//2:
      category_2 = category_2
    else:
      # add a random number to the category modulo n classes to ensure 2nd image 
      # has a different category
      category_2 = (category + rng.randint(1, n_classes)) % n_classes
    pairs[1][i,:,:,:] = X[category_2,idx_2].reshape(w, h, 1)

  return pairs, targets

In [53]:
def generate(batch_size, s="train"):
  ''' A generator for batches so model.fit_generator can be used '''
  while True:
    pairs, targets = get_batch(batch_size, s)
    yield(pairs, targets)

In [73]:
def make_oneshot_task(N, s="val", language=None):
  ''' Create pairs of test image, support set for testing N way one-shot learning'''
  if s == "train":
    X = Xtrain
    categories = train_classes
  else:
    X = Xval
    categories = val_classes
  n_classes, n_examples, w, h = X.shape

  indices = rng.randint(0, n_examples, size=(N,))
  if language is not None: # select characters if language is defined
    low, high = categories[language]
    if N > high - low:
      raise ValueError("This lanaguage ({}) has less than {} letters".format(language, N))

    categories = rng.choice(range(low, high), size=(N,), replace=False)
  else:
    categories = rng.choice(range(n_classes), size=(N,), replace=False)
  true_category = categories[0]
  ex1, ex2 = rng.choice(n_examples, replace=False, size=(2,))
  test_image = np.asarray([X[true_category, ex1, :, :]]*N).reshape(N, w, h, 1)
  support_set = X[categories, indices, :, :]
  support_set[0,:,:] = X[true_category, ex2]
  support_set = support_set.reshape(N, w, h, 1)
  targets = np.zeros((N,))
  targets[0] = 1
  targets, test_image, support_set = shuffle(targets, test_image, support_set)
  pairs = [test_image, support_set]

  return pairs,targets


In [69]:
def test_oneshot(model, N, k, s = "val", verbose = 0):
  ''' Test average N way oneshot learning accuracy of a siamese NN over k one shot tasks '''
  n_correct = 0
  if verbose:
    print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))

  for i in range(k):
    inputs, targets = make_oneshot_task(N,s)
    probs = model.predict(inputs)
    if np.argmax(probs) == np.argmax(targets):
      n_correct += 1
  percent_correct = (100.0 * n_correct / k)
  if verbose:
    print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
  return percent_correct

In [70]:

# Hyper parameters
evaluate_every = 200 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 20000 # No. of training iterations
N_way = 20 # how many classes for testing one-shot tasks
n_val = 250 # how many one-shot tasks to validate on
best = -1

In [71]:
!mkdir weights
model_path = 'saved_multi_ins_models/'

mkdir: cannot create directory ‘weights’: File exists


In [76]:
!mkdir saved_multi_ins_models

In [78]:
print("Start training process!")
print("----------------------------------")
t_start = time.time()
for i in range(1, n_iter+1):
  (inputs, targets) = get_batch(batch_size)
  loss = model.train_on_batch(inputs, targets)
  if i % evaluate_every == 0:
    print("\n ----------- \n")
    print("Time for {0} iterations: {1} mins".format(i, (time.time()- t_start)/60))
    print("Train_loss: {0}".format(loss))
    val_acc = test_oneshot(model, N_way, n_val, verbose=True)
    model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
    if val_acc >= best:
      print("Current best: {0}, previous best: {1}".format(val_acc, best))
      best=val_acc

Start training process!
----------------------------------

 ----------- 

Time for 200 iterations: 0.2537634611129761 mins
Train_loss: 0.7927048802375793
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 16.4% 20 way one-shot learning accuracy 

Current best: 16.4, previous best: -1

 ----------- 

Time for 400 iterations: 0.7037090023358663 mins
Train_loss: 0.7818618416786194
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 14.4% 20 way one-shot learning accuracy 


 ----------- 

Time for 600 iterations: 1.1598130583763122 mins
Train_loss: 0.7838113307952881
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 12.0% 20 way one-shot learning accuracy 


 ----------- 

Time for 800 iterations: 1.61709064245224 mins
Train_loss: 0.7521453499794006
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 12.0% 20 way one-shot learning accuracy 


 -------

In [ ]:
model.load_weights(os.path.join(model_path, "weights.20000.h5"))